**(a) Estratégia gulosa.** Considere a seguinte estratégia gulosa para resolver esse problema: “enquanto houver
espaço disponı́vel para plantar um pé de tomate, plante naquele que oferece a maior deliciosidade”. Lembre-se de que
um espaço só está disponı́vel para plantio se seus vizinhos imediatos não estiverem ocupados. Construa um
exemplo em que essa estratégia é ótima e outro em que ela não produz a solução ótima.

**Resposta:** Um exemplo de vetor T aonde esta estratégia descrita traz a solução ótima é o vetor 
$$T = [5, 1, 10, 1, 11]$$ 
Porque seguindo passo à passo, inicialmente encolhemos o valor 11 na última posição, então partimos para o 10 (como não há nenhum tomate adjacente, podemos plantar lá sem problemas) e depois para o 5 (que também está igualmente espaçado, portanto havendo a possibilidade de plantarmos um tomate na posição respectiva).  
Neste processo a deliciosidade total seria D = 11 + 10 + 5 = 26.  Que é a deliciosidade máxima possível a ser alcançada (podemos checar por exaustão, neste pequeno exemplo).

Um caso aonde esta estratégia não nos dá a solução ótima é quando temos, por exemplo, $$T = [6, 10, 8, 1].$$

Neste caso, temos que escolhemos plantar na posição 10, então nossa única alternativa é plantar na posição de valor 1, assim temos a deliciosidade total como D = 10 + 1 = 11.  Enquanto pode-se notar que plantar primeiro na posição de valor 8, então plantar na primeira posição de valor 6, encontramos uma deliciosidade total de D = 8 + 6 = 14, maior que a solução gulosa. 